In [1]:
import json
import re

import pandas as pd
pd.set_option('display.max_columns', None)
import requests
'''
LOOK THIS::: https://stackoverflow.com/questions/78307096/problem-how-to-scrape-dynamically-loaded-data-table-in-python
'''
url = "https://baseballsavant.mlb.com/leaderboard/statcast-park-factors"
url = "https://baseballsavant.mlb.com/leaderboard/statcast-park-factors?type=distance&year=2023&batSide=&stat=index_wOBA&condition=All&rolling="
url = "https://baseballsavant.mlb.com/leaderboard/statcast-park-factors?type=distance-all&year=2023&batSide=&stat=index_wOBA&condition=All&rolling="
response = requests.get(url)
data = re.search(r"data = (.*);", response.text).group(1)
data = json.loads(data)
df = pd.DataFrame(data)

engine = get_mlb_db_engine()

df.to_sql('ParkFactors', engine, if_exists='replace', index=False)

NameError: name 'get_mlb_db_engine' is not defined

In [157]:
cur_year = '2023'
ser = pd.Series(df.columns)

# Iterate over the items in the Series
for idx, item in enumerate(ser):
    if cur_year in item:
        print(idx, item)
        return idx



5 extra_distance_2023


In [ ]:
def get_park_factor(backtest_year=''):
    


In [11]:
#df['avg_extra_distance'] = df[df.columns[4:]].mean(axis=1)
import numpy as np
filled_df = df[df.columns[4:]].fillna(np.nan)
filled_df = filled_df.astype(float)
df['avg_extra_distance'] = filled_df.mean(axis=1)
df.sort_values(by='avg_extra_distance', ascending=False)[['venue_id','venue_name', 'avg_extra_distance']]                                         

,venue_id,venue_name,avg_extra_distance
18,19,Coors Field,20.011111
22,15,Chase Field,9.000000
25,5325,Globe Life Field,5.480000
24,12,Tropicana Field,4.588889
4,13,Choctaw Stadium,4.200000
21,7,Kauffman Stadium,3.100000
16,4705,Truist Park,1.987500
35,32,American Family Field,1.733333
15,2602,Great American Ball Park,1.077778
12,4169,loanDepot park,0.722222


In [41]:
from src.data.data_utils import query_mlb_db, get_mlb_db_engine

In [19]:
date_range_df = query_mlb_db('select min(game_date) as min_date, max(game_date) as max_date from Statcast')

In [29]:
from datetime import datetime

min_dt = datetime.strptime(date_range_df['min_date'][0], '%Y-%m-%d %H:%M:%S.%f')
min_date = min_dt.strftime('%m/%d/%Y')

max_dt = datetime.strptime(date_range_df['max_date'][0], '%Y-%m-%d %H:%M:%S.%f')
max_date = max_dt.strftime('%m/%d/%Y')

print(min_date, max_date)

04/02/2017 05/17/2024


In [51]:
game_pks_df = query_mlb_db('select distinct game_pk from Statcast')
game_pks = game_pks_df['game_pk']

#', '.join(game_pks)
str_game_pks = ', '.join(str(i) for i in game_pks)

In [88]:
import json
import requests

res = requests.get('https://statsapi.mlb.com/api/v1/schedule?gamePk=632097')
game_json = res.json()

In [89]:
game_json.keys()

dict_keys(['messageNumber', 'message', 'timestamp', 'traceId'])

In [87]:
game_json['dates'][0]['games'][0]['venue']

{'id': 3309, 'name': 'Nationals Park', 'link': '/api/v1/venues/3309'}

In [98]:
from tqdm import tqdm
import time

sleep_time = 1/10
venue_data = []
for game_pk in tqdm(game_pks, desc="Fetching Game Data", total=len(game_pks)):
    res = requests.get(f'https://statsapi.mlb.com/api/v1/schedule?gamePk={game_pk}')
    if res.status_code == 200:
        game_json = res.json()
        venue = game_json['dates'][0]['games'][0]['venue']
        venue['game_pk'] = game_pk
        venue_data.append(venue)
        #dont get rate limited... apparently max is 25/sec, 10/sec to be on safe side
        time.sleep(sleep_time)

Fetching Game Data: 100%|█████████████████████████████████████████████████████| 17360/17360 [1:22:42<00:00,  3.50it/s]


In [100]:
df = pd.DataFrame(venue_data)

In [102]:
df[['game_pk','name','id']]

,game_pk,name,id
0,490099,Busch Stadium,2889
1,490106,Tropicana Field,12
2,490110,Chase Field,15
3,490098,Citi Field,3289
4,490100,Globe Life Park in Arlington,13
...,...,...,...
17355,746641,Progressive Field,5
17356,746878,Wrigley Field,17
17357,747037,Oriole Park at Camden Yards,2
17358,747122,Truist Park,4705


In [103]:
engine = get_mlb_db_engine()

In [106]:
upload_df = df.rename(columns={"name": "venue_name", "id": "venue_id"})[['game_pk','venue_name','venue_id']]
upload_df

,game_pk,venue_name,venue_id
0,490099,Busch Stadium,2889
1,490106,Tropicana Field,12
2,490110,Chase Field,15
3,490098,Citi Field,3289
4,490100,Globe Life Park in Arlington,13
...,...,...,...
17355,746641,Progressive Field,5
17356,746878,Wrigley Field,17
17357,747037,Oriole Park at Camden Yards,2
17358,747122,Truist Park,4705


In [108]:
upload_df.to_sql('VenueGamePkMapping', engine, if_exists='replace', index=False)

17360

In [109]:
upload_df.to_csv('VenueMappingInit.csv')

In [4]:
from src.data.data_utils import query_mlb_db
df = query_mlb_db('Select * from VenueGamePkMapping')

In [5]:
df

,game_pk,venue_name,venue_id
0,490099,Busch Stadium,2889
1,490106,Tropicana Field,12
2,490110,Chase Field,15
3,490098,Citi Field,3289
4,490100,Globe Life Park in Arlington,13
...,...,...,...
17355,746641,Progressive Field,5
17356,746878,Wrigley Field,17
17357,747037,Oriole Park at Camden Yards,2
17358,747122,Truist Park,4705


In [12]:
backtest_yr = '2022'
park_factors_df = query_mlb_db('select * from ParkFactors')

In [29]:
#get 3 yr rolling average of park distance factors (not including backtest yr):
col_idx = None
for idx, col in enumerate(park_factors_df.columns):
    if backtest_yr in col:
        col_idx = idx
        break
# get average of past 3 years
park_factors_df['distance_factor'] = park_factors_df[park_factors_df.columns[col_idx:col_idx+3]].astype(float).mean(axis=1).fillna(0)

In [3]:
from datetime import datetime
datetime.now().year

2024

In [33]:
park_factors_df[park_factors_df.columns[4:7]].astype(float).mean(axis=1).fillna(0)

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
5      0.000000
6     -3.000000
7     -3.866667
8     -1.500000
9     -1.366667
10    -4.900000
11    -1.466667
12    -0.333333
13     0.733333
14    -2.233333
15     0.866667
16     3.666667
17    -0.366667
18    16.933333
19    -5.566667
20    -3.166667
21     3.966667
22    10.066667
23    -1.766667
24     4.866667
25     5.400000
26    -2.700000
27    -0.600000
28    -1.366667
29    -1.100000
30    -0.666667
31    -3.966667
32    -5.533333
33    -5.633333
34    -1.000000
35     2.566667
dtype: float64

In [31]:
park_factors_df[['venue_id', 'venue_name', 'distance_factor']].sort_values(by='distance_factor', ascending=False)

,venue_id,venue_name,distance_factor
18,19,Coors Field,20.166667
22,15,Chase Field,12.466667
25,5325,Globe Life Field,6.333333
24,12,Tropicana Field,4.233333
21,7,Kauffman Stadium,3.433333
16,4705,Truist Park,2.700000
15,2602,Great American Ball Park,1.500000
12,4169,loanDepot park,0.900000
9,3309,Nationals Park,0.066667
0,5381,London Stadium,0.000000


In [8]:
#url = "https://baseballsavant.mlb.com/leaderboard/statcast-park-factors?type=distance-all&year=2024&batSide=&stat=index_wOBA&condition=All&rolling="
url = f"https://baseballsavant.mlb.com/leaderboard/sprint_speed?min_season=2017&max_season={datetime.now().year}&position=&team=&min=10"
response = requests.get(url)
data = re.search(r"data = (.*);", response.text).group(1)
data = json.loads(data)
df = pd.DataFrame(data)
df[['runner_id', 'r_sprint_speed_top50percent_pretty']]

,runner_id,r_sprint_speed_top50percent_pretty
0,677951,30.4
1,687363,30.4
2,682829,30.3
3,607208,30.3
4,669352,30.3
...,...,...
478,596129,24.3
479,672386,24.2
480,607732,24.1
481,455117,24.0


In [10]:
from src.data.data_utils import query_mlb_db
test_df = query_mlb_db('select on_1b, on_2b, on_3b from Statcast where type="X"')

In [13]:
test_df.isnull().all(axis=1).sum() / len(test_df)


0.5641766692810928